# Export Hospital Level Dashboards

**Purpose:**

1. **Optionally authenticate** with `gcloud` (via user prompt).  
2. **Query BigQuery** for facility names (`faci_name` from `drg-viz.00_datasets.hci`).  
3. **(Optional) Save** these names to a local CSV for debugging.  
4. **Launch Selenium** (using `a_source_chrome_webdriver.py`) to open Power BI in **headless** mode,  
   select each facility in the slicer, and **export** a PDF to `./hospital-reports`.


In [ ]:
import os
import sys
import json

# Third-party imports
from google.cloud import bigquery
from google.api_core.exceptions import GoogleAPIError

# Local imports (the new approach)
project_root = os.path.expanduser("~/drg-hospital-dashboards")
if project_root not in sys.path:
    sys.path.append(project_root)

from src.functions import *  # now includes attempt_run and StepFailure

# Define the path for saving adaptive wait times
WAIT_TIMES_JSON = os.path.join("..", "data", "cache", "final_wait_times.json")

def main():
    """
    Full reload-from-scratch approach:
      1) Load WAIT_TIMES from JSON if it exists; otherwise, initialize.
         In TO_DEBUG mode, we want the wait times to persist (not reinitialize to 0 every time).
      2) Loop up to MAX_RETRIES times.
      3) Each loop calls attempt_run().
      4) If attempt_run() succeeds, we break.
         Otherwise, attempt_run() raises StepFailure(step_name).
      5) The failing step’s WAIT_TIMES is incremented by 1 (if configured), then we re-run.
      6) If MAX_RETRIES is exceeded, we stop.
      7) If TO_DEBUG is True and the run is successful, save final wait times.
         If TO_DEBUG is False, failed hospitals are appended to FAILED_HOSPITALS.
    """
    
    # Load wait times if the JSON exists; otherwise, initialize.
    if os.path.isfile(WAIT_TIMES_JSON):
        with open(WAIT_TIMES_JSON, "r") as f:
            loaded_times = json.load(f)
        WAIT_TIMES.update(loaded_times)
        print(f"Loaded WAIT_TIMES from {WAIT_TIMES_JSON}: {WAIT_TIMES}")
    else:
        # Initialize to zeros if file does not exist.
        WAIT_TIMES.clear()
        WAIT_TIMES.update({
            "iframe_wait": 0,
            "dropdown_sleep": 0,
            "search_sleep": 0,
            "visual_update_sleep": 0,
            "webdriver_wait_first_result": 0
        })
        print("No existing wait times found; initializing WAIT_TIMES to zeros.")

    # In non-debug mode, we load the wait times (if available) but TO_DEBUG mode uses the same logic.
    attempt_count = 1
    while attempt_count <= MAX_RETRIES:
        print(f"\n=== Attempt #{attempt_count} (TO_DEBUG={TO_DEBUG}) with WAIT_TIMES={WAIT_TIMES} ===")
        try:
            # Attempt the complete process
            success = attempt_run()
            if success:
                print("Run succeeded! Stopping.")
                break
        except StepFailure as sf:
            step = sf.step_name
            # Only increment wait time if this step is configured in WAIT_TIMES
            if step in WAIT_TIMES:
                WAIT_TIMES[step] += 1
                print(f"Run failed at step '{step}'. Increased WAIT_TIMES['{step}'] to {WAIT_TIMES[step]}.")
            else:
                print(f"Run failed at step '{step}', but no adaptive wait time is configured for it. Restarting process.")
            print("Will reload from scratch...\n")
        except Exception as e:
            # Any unexpected error aborts the attempt
            print(f"Unexpected error: {e}")
            print("Aborting this attempt.")
            break

        attempt_count += 1

    if attempt_count > MAX_RETRIES:
        print(f"Gave up after {MAX_RETRIES} attempts.")
        print(f"Final WAIT_TIMES: {WAIT_TIMES}")
    else:
        print(f"Success on attempt #{attempt_count}. Final WAIT_TIMES: {WAIT_TIMES}")

    # Save final results
    if TO_DEBUG:
        # Save final wait times to JSON (these will be used on subsequent runs)
        os.makedirs(os.path.dirname(WAIT_TIMES_JSON), exist_ok=True)
        with open(WAIT_TIMES_JSON, "w") as f:
            json.dump(WAIT_TIMES, f)
        print(f"Saved final WAIT_TIMES to {WAIT_TIMES_JSON}.")
    else:
        print(f"Processing completed with wait times: {WAIT_TIMES}")
        # In non-debug mode, failed hospitals are saved in the worker task logic (see attempt_run below)

if __name__ == "__main__":
    main()
